`Pair Programming Limpieza V Sklearn `

En este caso trabajaremos con el dataframe que limpiamos en el ejercicio de pair de Limpieza III donde limpiabamos las columnas de species, age, etc. y eliminabamos los outliers. Hoy volveremos a gestionar valores nulos, pero en este caso usaremos los métodos de imputación de sklearn.

In [43]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
df=pd.read_csv('../Datos/attacks_limpieza_III.csv')
df.head(2)

,case_number,year,type,country,area,location,activity,name,time,species,siglo,species_,injury,mes,fatal_bien,genero_bien,categorias_tiburones,edades
0,2018.06.25,2018,Boating,usa,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,18h00,White shark,siglo - XXI,NaN,"No injury to occupant, outrigger canoe and pad...",Jun,N,F,White,57.0
1,2018.06.03.a,2018,Unprovoked,brazil,Pernambuco,"Piedade Beach, Recife",Swimming,Jose Ernesto da Silva,Late afternoon,Tiger shark,siglo - XXI,NaN,FATAL,Jun,Y,M,Tiger,18.0


1. Es el momento de eliminar los nulos:
Reemplazad los valores nulos de la columna age por la media de la edad usando el método SimpleImputer.

In [44]:
imputer = SimpleImputer(strategy='mean', missing_values=np.nan)
imputer = imputer.fit(df[['edades']])
df['edades']=imputer.transform(df[['edades']])

In [45]:
df['edades']

0       57.000000
1       18.000000
2       15.000000
3       32.000000
4       21.000000
          ...    
1667    27.889036
1668    27.889036
1669    27.889036
1670    27.889036
1671    15.000000
Name: edades, Length: 1672, dtype: float64

Reemplazad los valores nulos de la columna sex por la moda, usando el método SimpleImputer.
💡 Pista 💡 La moda en este tipo de aproximación se indica como most_frequent.

In [46]:
imputer = SimpleImputer(strategy='most_frequent', missing_values=np.nan)
imputer = imputer.fit(df[['genero_bien']])
df['genero_bien']=imputer.transform(df[['genero_bien']])

In [47]:
df['genero_bien']

0       F
1       M
2       M
3       M
4       M
       ..
1667    M
1668    M
1669    M
1670    M
1671    M
Name: genero_bien, Length: 1672, dtype: object

Reemplazad los valores nulos de la columna type por el valor más frecuente (la moda) con el método SimpleImputer.

In [48]:
imputer = SimpleImputer(strategy='most_frequent', missing_values=np.nan)
imputer = imputer.fit(df[['type']])
df['type']=imputer.transform(df[['type']])

In [49]:
df['type']

0          Boating
1       Unprovoked
2       Unprovoked
3         Provoked
4          Invalid
           ...    
1667    Unprovoked
1668    Unprovoked
1669    Unprovoked
1670    Unprovoked
1671    Unprovoked
Name: type, Length: 1672, dtype: object

In [50]:
#decidimos hacerlo también en la columna de categorias_tiburones

imputer = SimpleImputer(strategy='most_frequent', missing_values=np.nan)
imputer = imputer.fit(df[['categorias_tiburones']])
df['categorias_tiburones']=imputer.transform(df[['categorias_tiburones']])

Utilizad el método KNN Imputer para reemplazar todos los valores nulos de las columnas numéricas.

In [51]:
numericas = df.select_dtypes(include = np.number)

In [52]:
imputerKNN = KNNImputer(n_neighbors=5)
imputerKNN.fit(numericas)

KNNImputer()

In [53]:
numericas_knn= imputerKNN.transform(numericas)

In [54]:
df_knn_imputer = pd.DataFrame(numericas_knn, columns = numericas.columns)

In [55]:
df_knn_imputer.head()

,year,edades
0,2018.0,57.0
1,2018.0,18.0
2,2018.0,15.0
3,2018.0,32.0
4,2018.0,21.0


In [56]:
columnas_knn = df_knn_imputer.columns

In [57]:
df.head(2)

,case_number,year,type,country,area,location,activity,name,time,species,siglo,species_,injury,mes,fatal_bien,genero_bien,categorias_tiburones,edades
0,2018.06.25,2018,Boating,usa,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,18h00,White shark,siglo - XXI,NaN,"No injury to occupant, outrigger canoe and pad...",Jun,N,F,White,57.0
1,2018.06.03.a,2018,Unprovoked,brazil,Pernambuco,"Piedade Beach, Recife",Swimming,Jose Ernesto da Silva,Late afternoon,Tiger shark,siglo - XXI,NaN,FATAL,Jun,Y,M,Tiger,18.0


Utilizad el método Iterative Imputer para reemplazar todos los valores nulos de las columnas numéricas.
¿Podríais explicar qué diferencia hay entre estos dos últimos métodos?

In [58]:
imputer = IterativeImputer(n_nearest_features=None, imputation_order='ascending')
imputer.fit(numericas)
imputer.transform(numericas)
numericas_trans = pd.DataFrame(imputer.transform(numericas), columns = numericas.columns)
numericas_trans.head()

,year,edades
0,2018.0,57.0
1,2018.0,18.0
2,2018.0,15.0
3,2018.0,32.0
4,2018.0,21.0


In [59]:
numericas_trans.isnull().sum()

year      0
edades    0
dtype: int64

In [60]:
columnas = numericas_trans.columns
df.drop(columnas, axis = 1, inplace = True)
df[columnas] = numericas_trans[columnas]
df.head(2)

,case_number,type,country,area,location,activity,name,time,species,siglo,species_,injury,mes,fatal_bien,genero_bien,categorias_tiburones,year,edades
0,2018.06.25,Boating,usa,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,18h00,White shark,siglo - XXI,NaN,"No injury to occupant, outrigger canoe and pad...",Jun,N,F,White,2018.0,57.0
1,2018.06.03.a,Unprovoked,brazil,Pernambuco,"Piedade Beach, Recife",Swimming,Jose Ernesto da Silva,Late afternoon,Tiger shark,siglo - XXI,NaN,FATAL,Jun,Y,M,Tiger,2018.0,18.0


In [61]:
nulos = pd.DataFrame((df.isnull().sum() * 100) / df.shape[0]).reset_index()
nulos.columns = ["columna", "porcentaje"]
nulos

,columna,porcentaje
0,case_number,0.000000
1,type,0.000000
2,country,0.598086
3,area,2.751196
4,location,3.050239
5,activity,1.854067
6,name,0.538278
7,time,9.569378
8,species,10.167464
9,siglo,10.167464


2.Guardad el csv para seguir trabajando con el en los siguientes ejercicios de pair

In [62]:
df.to_csv('../Datos/attacks_limpieza_V.csv', index=False)